# Introduction

Neural networks can also be used for prediction.

In this exercise we use a part of the signal to predict the next signal.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Generate synthetic sinusoidal data
t = np.linspace(0, 100, 1000)
signal = np.sin(t)

# Exercises

## Exercise 1

Create a window function which moves over the signal with a window size of 150.

The input should this be 150 values and the output is as well because we are using a part of the signal to predict the next part.

The shape should be (1700, 150).

In [ ]:
sequences = []
labels = []
window_size = 150
for i in range(len(signal)):
    if i < (len(signal) - 2 * window_size):
        window = signal[i:i + window_size]
        next_window = signal[i + window_size: i + 2 * window_size]
        sequences.append(window)
        labels.append(next_window)

sequences = [e for e in sequences if len(e) == window_size]
labels = [e for e in labels if len(e) == window_size]

sequences = np.array(sequences)
labels = np.array(labels)

In [ ]:
plt.figure(figsize=(14, 7))
plt.title('Example of the signal')
plt.plot(signal[0:window_size], label='True Sinusoid', color='blue')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

## Exercise 2

What should the activation function be?

Could we use another activation function if we normalized between 0 and 1?

In [ ]:
# Split the data into training and test sets
train_size = int(0.8 * len(sequences))
train_sequences = sequences[:train_size]
train_labels = labels[:train_size]
test_sequences = sequences[train_size:]
test_labels = labels[train_size:]

# Reshape the data for RNN
train_sequences = np.reshape(train_sequences, (train_sequences.shape[0], train_sequences.shape[1], 1))
test_sequences = np.reshape(test_sequences, (test_sequences.shape[0], test_sequences.shape[1], 1))

# Build the RNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(50, activation='relu', return_sequences=True, input_shape=(window_size, 1)),
    tf.keras.layers.SimpleRNN(50, activation='relu'),
    tf.keras.layers.Dense(window_size, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(train_sequences, train_labels, epochs=10, batch_size=32, validation_data=(test_sequences, test_labels))

## Exercise 3

Predict the next part of the signal.

In [ ]:
# Predict on test data
predictions = model.predict(test_sequences)
predictions.shape

In [ ]:
# Plot the true sinusoidal function and predictions
true_signal = test_sequences[0].reshape(-1)
predicted_next_part_of_signal = np.append(true_signal, predictions[0].reshape(-1))
plt.figure(figsize=(14, 7))
plt.title('Sinusoidal Function Prediction using RNN')
plt.plot(true_signal, label='True Sinusoid', color='blue')
plt.plot(predicted_next_part_of_signal, label='Predicted Sinusoid', color='red', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

# Bonus

Use prediction to generate new prediction.

Determine what the shape of the testdata was and try to reshape it such that it fits in the `predict` function.

In [ ]:
# Predict on test data
next_prediction = model.predict(predictions[0].reshape(1,window_size,1))
next_prediction.shape

In [ ]:
# Plot the true sinusoidal function and predictions
true_signal = test_sequences[0].reshape(-1)
predicted_next_part_of_signal = np.append(true_signal, predictions[0].reshape(-1))
predicted_next_next_part_of_signal = np.append(predicted_next_part_of_signal, next_prediction)
plt.figure(figsize=(14, 7))
plt.title('Sinusoidal Function Prediction using using previous prediction')
plt.plot(true_signal, label='True Sinusoid', color='blue')
plt.plot(predicted_next_part_of_signal, label='Predicted Sinusoid', color='red', linestyle='--')
plt.plot(predicted_next_next_part_of_signal, label='Predicted Sinusoid using prediction', color='green', linestyle='dotted')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()